In [1]:
import pandas as pd
import censusdata
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)

In [2]:
# Check the ACS lookup table here:
# https://www.census.gov/programs-surveys/acs/technical-documentation/summary-file-documentation.2015.html
# Targeting data: income, population, age, gender, and race
# Relevant tables: B00001, B00002, B01001, B01002, B02001, B06011
# censusdata.search('acs5', 2015, 'label', 'Block Group')
# censusdata.search('acs5', 2015, 'label', 'Census Tract')
# censusdata.search('acs5', 2015, 'label', 'Block')

In [3]:
# check vars here.
censusdata.printtable(censusdata.censustable('acs5', 2015, 'B06010'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B06010_001E  | B06010. Place of Birth by Indi | Total:                                                   | int  
B06010_002E  | B06010. Place of Birth by Indi | No income                                                | int  
B06010_003E  | B06010. Place of Birth by Indi | With income:                                             | int  
B06010_004E  | B06010. Place of Birth by Indi | !! With income: $1 to $9,999 or loss                     | int  
B06010_005E  | B06010. Place of Birth by Indi | !! With income: $10,000 to $14,999                       | int  
B06010_006E  | B06010. Place of Birth by Indi | !! With income: $15,000 to $24,999                       | int  
B06010_007E  | B06010. Place of Birth by Indi | !! With income: $25,000 to $34,999           

In [4]:
# variable list
var_list = ['B00001_001E', 'B00002_001E', 
            'B01001_001E', 'B01001_002E', 'B01001_026E', 
            'B01002_001E', 
            'B02001_001E', 'B02001_002E', 'B02001_003E', 'B02001_004E', 'B02001_005E', 
            'B06010_001E', 'B06010_002E', 'B06010_003E', 'B06010_004E', 'B06010_005E', 'B06010_006E', 'B06010_007E', 'B06010_008E', 'B06010_009E', 'B06010_010E', 'B06010_011E']

var_names = ['total_pop', 'total_houses', 
             'total_sex', 'male', 'female',
             'median_age',
             'total_race', 'white', 'black', 'native', 'asian',
             'total_pop_inc', 'pop_no_inc', 'pop_with_inc', 'pop_inc_10k', 'pop_inc_1k_15k', 'pop_inc_15k_25k', 'pop_inc_25k_35k', 'pop_inc_35k_50k', 'pop_inc_50k_65k', 'pop_inc_65k_75k', 'pop_inc_75k']


In [57]:
# cook data
cook_tract = censusdata.download('acs5', 2015,
                                 censusdata.censusgeo([('state', '17'), ('county', '031'), ('tract', '*')]),
                                 var_list)

In [58]:
print(cook_tract.shape)

(1319, 22)


In [59]:
# replace names
cook_tract.columns = var_names

# clean cook county data
state_fips=['17']*cook_tract.shape[0]
county_fips=['031']*cook_tract.shape[0]
tract_fips=[]
full_tract_fips=[]

for i in range(cook_tract.shape[0]):
    tract_fips.append(cook_tract.index[i].params()[2][1])
    full_tract_fips.append('17'+'031'+cook_tract.index[i].params()[2][1])

cook_tract['state_fips']=state_fips
cook_tract['county_fips']=county_fips
cook_tract['tract_fips']=tract_fips
cook_tract['full_tract_fips']=full_tract_fips

# reset index
cook_tract.reset_index(drop = True, inplace = True)
cook_tract

,total_pop,total_houses,total_sex,male,female,median_age,total_race,white,black,native,...,pop_inc_1k_15k,pop_inc_15k_25k,pop_inc_25k_35k,pop_inc_35k_50k,pop_inc_50k_65k,pop_inc_65k_75k,pop_inc_75k,state_fips,county_fips,tract_fips
0,334,121,7751,3770,3981,45.6,7751,562,6770,3,...,564,711,544,819,637,194,748,17,031,700501
1,245,79,3162,1598,1564,43.4,3162,107,2816,0,...,211,177,190,241,224,143,485,17,031,700502
2,149,75,1290,605,685,28.9,1290,66,1224,0,...,106,119,52,59,25,26,21,17,031,710100
3,286,148,5397,2408,2989,31.5,5397,143,5146,0,...,425,651,463,232,118,55,52,17,031,710200
4,152,77,1301,542,759,39.8,1301,0,1268,4,...,112,241,97,47,51,10,32,17,031,710300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314,452,137,6785,3136,3649,32.0,6785,2243,930,40,...,348,721,794,475,360,150,226,17,031,700200
1315,423,133,6151,3184,2967,33.6,6151,3237,678,53,...,367,746,604,607,374,103,198,17,031,700301
1316,409,140,5314,2699,2615,34.8,5314,3270,918,6,...,336,616,601,339,253,206,429,17,031,700302
1317,442,147,6153,3181,2972,32.6,6153,1332,3665,13,...,488,484,453,585,568,221,424,17,031,700401


In [60]:
cook_tract.shape

(1319, 25)

In [20]:
# state number list
state_df = pd.read_csv('./data_usa/state_fips.csv')
state_dir = {}
for key,item in zip(state_df['Postal Code'], state_df['FIPS']):
    # first, control the digits
    item = str(item)
    if len(str(item)) == 1:
        item = '0'+str(item)
    # 
    if str(item) not in ['60','66','69','72','78','02','15']:
        state_dir[item] = key # ahhh...the order sucks
        
state_dir
# state_dic = []

{'01': 'AL',
 '04': 'AZ',
 '05': 'AR',
 '06': 'CA',
 '08': 'CO',
 '09': 'CT',
 '10': 'DE',
 '12': 'FL',
 '13': 'GA',
 '16': 'ID',
 '17': 'IL',
 '18': 'IN',
 '19': 'IA',
 '20': 'KS',
 '21': 'KY',
 '22': 'LA',
 '23': 'ME',
 '24': 'MD',
 '25': 'MA',
 '26': 'MI',
 '27': 'MN',
 '28': 'MS',
 '29': 'MO',
 '30': 'MT',
 '31': 'NE',
 '32': 'NV',
 '33': 'NH',
 '34': 'NJ',
 '35': 'NM',
 '36': 'NY',
 '37': 'NC',
 '38': 'ND',
 '39': 'OH',
 '40': 'OK',
 '41': 'OR',
 '42': 'PA',
 '44': 'RI',
 '45': 'SC',
 '46': 'SD',
 '47': 'TN',
 '48': 'TX',
 '49': 'UT',
 '50': 'VT',
 '51': 'VA',
 '53': 'WA',
 '54': 'WV',
 '55': 'WI',
 '56': 'WY'}

In [21]:
# us tract data
us_tract = {}

for key in state_dir.keys():
    print(state_dir[key])
    us_tract[key] = censusdata.download('acs5', 2015,
                                 censusdata.censusgeo([('state', key), ('tract', '*')]),
                                 var_list)

AL
AZ
AR
CA
CO
CT
DE
FL
GA
ID
IL
IN
IA
KS
KY
LA
ME
MD
MA
MI
MN
MS
MO
MT
NE
NV
NH
NJ
NM
NY
NC
ND
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VT
VA
WA
WV
WI
WY


In [87]:
us_tract_df = pd.concat(us_tract.values())

In [88]:
# replace names
us_tract_df.columns = var_names

# clean cook county data
state_fips=[]
county_fips=[]
tract_fips=[]
full_tract_fips=[]

for i in range(us_tract_df.shape[0]):
    state_fips.append(us_tract_df.index[i].params()[0][1])
    county_fips.append(us_tract_df.index[i].params()[1][1])
    tract_fips.append(us_tract_df.index[i].params()[2][1])
    full_tract_fips.append(us_tract_df.index[i].params()[0][1]+us_tract_df.index[i].params()[1][1]+us_tract_df.index[i].params()[2][1])

us_tract_df['state_fips']=state_fips
us_tract_df['county_fips']=county_fips
us_tract_df['tract_fips']=tract_fips
us_tract_df['full_tract_fips']=full_tract_fips

# reset index
us_tract_df.reset_index(drop = True, inplace = True)
us_tract_df


,total_pop,total_houses,total_sex,male,female,median_age,total_race,white,black,native,...,pop_inc_15k_25k,pop_inc_25k_35k,pop_inc_35k_50k,pop_inc_50k_65k,pop_inc_65k_75k,pop_inc_75k,state_fips,county_fips,tract_fips,full_tract_fips
0,341.0,126.0,5143,2180,2963,39.0,5143,4036,884,31,...,690,519,547,400,133,211,01,097,006501,01097006501
1,331.0,133.0,11578,5705,5873,37.6,11578,10678,451,0,...,1115,737,1287,863,277,1181,01,097,006502,01097006502
2,266.0,101.0,5574,2788,2786,38.9,5574,5033,419,12,...,785,520,425,418,138,283,01,097,006600,01097006600
3,318.0,128.0,6436,3174,3262,39.8,6436,4799,883,11,...,993,363,340,504,53,315,01,097,006701,01097006701
4,231.0,112.0,3270,1564,1706,39.0,3270,2332,415,0,...,469,414,357,188,0,92,01,097,006702,01097006702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72354,318.0,189.0,1797,906,891,37.6,1797,1720,17,0,...,116,184,95,150,51,339,56,037,971600,56037971600
72355,354.0,169.0,4830,2335,2495,48.7,4830,4731,27,8,...,812,382,520,514,310,710,56,039,967600,56039967600
72356,193.0,85.0,2396,1201,1195,32.2,2396,2170,7,7,...,287,408,250,338,79,125,56,039,967701,56039967701
72357,245.0,161.0,7099,4041,3058,36.8,7099,6466,0,4,...,956,572,916,744,176,873,56,039,967702,56039967702


In [89]:
us_tract_df.shape

(72359, 26)

### Save files

In [90]:
import pickle

with open('../../data/raw_data/cook_tract.pickle', 'wb') as f:
    pickle.dump(cook_tract, f)

with open('../../data/raw_data/usa_tract.pickle', 'wb') as f:
    pickle.dump(us_tract_df, f)
    

In [91]:
# with open('./data_usa/cook_tract.pickle', 'rb') as f:
#     cook_read = pickle.load(f)
